In [48]:
# Import our dependencies
#!pip install psycopg2
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import plotly.express as px
from matplotlib import pyplot
import hvplot.pandas
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import statistics
from matplotlib.ticker import MultipleLocator
import scipy.stats as sts
import time
import re
from sqlalchemy import create_engine 
import sqlalchemy
import json
import psycopg2
from config import db_password
!pip install ipython-sql
from sklearn.neighbors import KNeighborsClassifier


In [49]:
# Define variable for database connection
protocol = "postgres"
user = "postgres"
location = "localhost"
port = "5432"
db = "Time_Series"

In [50]:
connection = f'{protocol}://{user}:{db_password}@{location}:{port}/{db}'
engine = create_engine(connection)


In [51]:
# Test and verify that Jupyter Notebook is connected and able to read datafrom from PostgresDB

with engine.connect() as connection:
    result = connection.execute("select * from stocks limit 5")
    for row in result:
       print("Name:", row['name_'])
        

Name: AAL
Name: AAL
Name: AAL
Name: AAL
Name: AAL


In [52]:
%load_ext sql
 
#%%sql 
#SELECT * FROM stocks FETCH FIRST ROW ONLY;

#select * from stocks 
#SELECT * FROM stocks LIMIT 5;

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [53]:
stocks_df = pd.read_sql_table('stocks',engine)
stocks_df.head()

,date,open_,high,low,close_,volume,name_
0,2/8/2013,15.07,15.12,14.63,14.75,8407500,AAL
1,2/11/2013,14.89,15.01,14.26,14.46,8882000,AAL
2,2/12/2013,14.45,14.51,14.1,14.27,8126000,AAL
3,2/13/2013,14.3,14.94,14.25,14.66,10259500,AAL
4,2/14/2013,14.94,14.96,13.16,13.99,31879900,AAL


In [54]:
# Generate our categorical variable list
stocks_cat = stocks_df.dtypes[stocks_df.dtypes == "object"].index.tolist()

display(stocks_cat)

['date', 'open_', 'high', 'low', 'close_', 'volume', 'name_']

In [55]:
# Check the number of unique values in each column
stocks_df[stocks_cat].nunique()

date      1259
open_     3264
high      3353
low       3344
close_    3318
volume    3775
name_        3
dtype: int64

In [56]:
# Get the unique values of the names in Name.
stocks_df["name_"].unique()


array(['AAL', 'AAPL', 'AAP'], dtype=object)

In [57]:
stocks_df2 = stocks_df.set_index(pd.to_datetime(stocks_df['date'].values))

stocks_df2["date"] = (pd.to_datetime(stocks_df['date'].values))

stocks_df2.drop(columns=['date'], inplace=True)
 
stocks_df2=stocks_df2.dropna(how="all")

stocks_df2.head()

,open_,high,low,close_,volume,name_
2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2013-02-12,14.45,14.51,14.1,14.27,8126000,AAL
2013-02-13,14.3,14.94,14.25,14.66,10259500,AAL
2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [58]:
# Drop Nan and nulls
stocks_df=stocks_df.dropna(how="all")
stocks_df.head()

,date,open_,high,low,close_,volume,name_
0,2/8/2013,15.07,15.12,14.63,14.75,8407500,AAL
1,2/11/2013,14.89,15.01,14.26,14.46,8882000,AAL
2,2/12/2013,14.45,14.51,14.1,14.27,8126000,AAL
3,2/13/2013,14.3,14.94,14.25,14.66,10259500,AAL
4,2/14/2013,14.94,14.96,13.16,13.99,31879900,AAL


In [60]:
# Check to see if there are null values or missing data. This will show where there is empty
for column in stocks_df.columns:
    print(f"Column{column} has {stocks_df[column].isnull().sum()} null values" )

Columndate has 0 null values
Columnopen_ has 0 null values
Columnhigh has 0 null values
Columnlow has 0 null values
Columnclose_ has 0 null values
Columnvolume has 0 null values
Columnname_ has 0 null values


In [61]:
X_df = pd.get_dummies(stocks_df[["name_", "open_", "close_", "volume"]])

X_df.head()

,name__AAL,name__AAP,name__AAPL,open__100,open__100.05,open__100.09,open__100.15,open__100.27,open__100.29,open__100.32,...,volume_996380,volume_996410,volume_997320,volume_99756489,volume_997998,volume_9981485,volume_9991117,volume_9994162,volume_9997306,volume_9997418
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Standardize data with StandardScaler
stocks_scaled = StandardScaler().fit_transform(X_df)
print(stocks_scaled)

[[ 1.41421356 -0.70710678 -0.70710678 ... -0.01627361 -0.01627361
  -0.01627361]
 [ 1.41421356 -0.70710678 -0.70710678 ... -0.01627361 -0.01627361
  -0.01627361]
 [ 1.41421356 -0.70710678 -0.70710678 ... -0.01627361 -0.01627361
  -0.01627361]
 ...
 [-0.70710678  1.41421356 -0.70710678 ... -0.01627361 -0.01627361
  -0.01627361]
 [-0.70710678  1.41421356 -0.70710678 ... -0.01627361 -0.01627361
  -0.01627361]
 [-0.70710678  1.41421356 -0.70710678 ... -0.01627361 -0.01627361
  -0.01627361]]


In [62]:
# Initialize PCA model
pca = PCA(n_components=3)

# Apply dimensionality reduction on the scaled data
# Get three principal components 
stocks_scaled_pca = pca.fit_transform(stocks_scaled)

# Transform PCA data into a DataFrame. This will contain the three principal components
stocks_pcs_df = pd.DataFrame(stocks_scaled_pca, columns=["open_", "close_", "volume"], index=stocks_df.index)


stocks_pcs_df.head(10)

,open_,close_,volume
0,1.757662,0.059696,-1.404313
1,3.271592,0.080312,3.001967
2,2.272642,-2.056511,-0.064783
3,0.468286,1.755687,0.377530
4,2.307647,-2.142412,-0.621440
5,2.647571,-1.653134,1.027079
6,3.383863,-0.209001,0.831796
7,1.866378,-0.052032,0.220887
8,2.235617,0.775982,1.302952
9,4.694760,-2.184683,2.370442


In [63]:
# Use explained_variance_ratio to learn how much information can be attributed to each
# principal component
pca.explained_variance_ratio_

array([0.00038953, 0.0003625 , 0.00028942])

In [64]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
stocks_encode_df = pd.DataFrame(enc.fit_transform(stocks_df[stocks_cat]))

# Add the encoded variable names to the DataFrame
stocks_encode_df.columns = enc.get_feature_names(stocks_cat)
stocks_encode_df.head()

,date_1/10/2014,date_1/10/2017,date_1/10/2018,date_1/11/2016,date_1/11/2017,date_1/11/2018,date_1/12/2015,date_1/12/2016,date_1/12/2017,date_1/12/2018,...,volume_99756489,volume_997998,volume_9981485,volume_9991117,volume_9994162,volume_9997306,volume_9997418,name__AAL,name__AAP,name__AAPL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [25]:
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
	km = KMeans(n_clusters=i, random_state=0)
	km.fit(stocks_pcs_df)
	inertia.append(km.inertia_)

# Create the elbow curve
stocks_elbow_data = {"k": k, "inertia": inertia}
stocks_df_elbow = pd.DataFrame(stocks_elbow_data)

stocks_df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [65]:
# Initialize the K-means model
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(stocks_pcs_df)

# Predict clusters
predictions = model.predict(stocks_pcs_df)

stocks_pcs_df["open_"] = stocks_df["open_"]
stocks_pcs_df["close_"] = stocks_df["close_"]
stocks_pcs_df["volume"] = stocks_df["volume"]
stocks_pcs_df["name_"] = stocks_df["name_"]

# Add the predicted class columns
stocks_pcs_df["class"] = model.labels_
stocks_pcs_df.head()

,open_,close_,volume,name_,class
0,15.07,14.75,8407500,AAL,1
1,14.89,14.46,8882000,AAL,1
2,14.45,14.27,8126000,AAL,1
3,14.3,14.66,10259500,AAL,0
4,14.94,13.99,31879900,AAL,1


In [66]:
# Merge one-hot encoded features and drop the originals
stocks_df = stocks_df.merge(stocks_encode_df,left_index=True, right_index=True)

stocks_df = stocks_df.drop(stocks_cat,1)

stocks_df.head()

,date_1/10/2014,date_1/10/2017,date_1/10/2018,date_1/11/2016,date_1/11/2017,date_1/11/2018,date_1/12/2015,date_1/12/2016,date_1/12/2017,date_1/12/2018,...,volume_99756489,volume_997998,volume_9981485,volume_9991117,volume_9994162,volume_9997306,volume_9997418,name__AAL,name__AAP,name__AAPL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [67]:
# For inproved visualization of the plot, we scale with MinMaxScaler() and transform
# with fit_transform


mm_scaler = MinMaxScaler()
stocks_plot_data = mm_scaler.fit_transform(stocks_df2[["open_", "close_", "volume"]])

stocks_plot_df = pd.DataFrame(
    stocks_plot_data, columns=["open_","close_","volume"],index=stocks_df2.index)

stocks_plot_df["volume"] = stocks_pcs_df["volume"]
stocks_plot_df["class"] = stocks_pcs_df["class"]

stocks_plot_df=stocks_plot_df.dropna(how="all")

stocks_plot_df.head()

,open_,close_,volume,class
2013-02-08,0.010260,0.009234,NaN,NaN
2013-02-11,0.009304,0.007686,NaN,NaN
2013-02-12,0.006964,0.006672,NaN,NaN
2013-02-13,0.006167,0.008753,NaN,NaN
2013-02-14,0.009569,0.005177,NaN,NaN


In [68]:
# Create a dataframe for a one yr block (2016) using loc
oneyr16_stocks = stocks_df2.loc['2016-01-01':'2016-12-31']
oneyr16_stocks_df = pd.DataFrame(oneyr16_stocks)
oneyr16_stocks_df.head()

,open_,high,low,close_,volume,name_
2016-01-04,41.29,41.3501,40.3,40.91,12037151,AAL
2016-01-05,41.18,41.49,39.95,40.52,10514164,AAL
2016-01-06,40.05,41.7,40,41.23,12393780,AAL
2016-01-07,40.97,41.52,39.68,40.45,11312876,AAL
2016-01-08,40.94,41.36,40.26,40.37,8853096,AAL


In [69]:
# Arrange the data in separate order for each stock ticker name (AAL, AAP, AAPL) from 2016 chunk
sample_opening = oneyr16_stocks_df[oneyr16_stocks_df['name_'].isin(['AAL','AAP','AAPL'])]

sample_opening_df = pd.DataFrame(sample_opening)

sample_opening_df.head()

,open_,high,low,close_,volume,name_
2016-01-04,41.29,41.3501,40.3,40.91,12037151,AAL
2016-01-05,41.18,41.49,39.95,40.52,10514164,AAL
2016-01-06,40.05,41.7,40,41.23,12393780,AAL
2016-01-07,40.97,41.52,39.68,40.45,11312876,AAL
2016-01-08,40.94,41.36,40.26,40.37,8853096,AAL


In [96]:
# Create a dataframe for AAPL stock price for the year 2016
aapl_2016 = oneyr16_stocks_df[oneyr16_stocks_df['name_'].isin(['AAPL'])]
aapl_2016_df = pd.DataFrame(aapl_2016)

aapl_2016_df.head()

,open_,high,low,close_,volume,name_
2016-01-04,102.61,105.368,102,105.35,67649387,AAPL
2016-01-05,105.75,105.85,102.41,102.71,55790992,AAPL
2016-01-06,100.56,102.37,99.87,100.7,68457388,AAPL
2016-01-07,98.68,100.13,96.43,96.45,81094428,AAPL
2016-01-08,98.55,99.11,96.76,96.96,70798016,AAPL


In [98]:
# Create a dataframe for AAP stock price for the year 2016
aap_2016 = oneyr16_stocks_df[oneyr16_stocks_df['name_'].isin(['AAP'])]
aap_2016_df = pd.DataFrame(aap_2016)

aap_2016_df.head()

,open_,high,low,close_,volume,name_
2016-01-04,149.25,154.48,147.51,152.24,1935307,AAP
2016-01-05,152.35,153.79,149.89,151.2,1127677,AAP
2016-01-06,149.68,150.05,145.78,147.2,1048419,AAP
2016-01-07,144.8,149.4599,143.13,148.83,1343674,AAP
2016-01-08,149.03,149.44,144.871,145.56,1030364,AAP


In [99]:
# Create a dataframe for closing data for AAP
aap_closing2016_df = pd.DataFrame(aap_2016_df.close_)
aap_closing2016_df.head()

,close_
2016-01-04,152.24
2016-01-05,151.2
2016-01-06,147.2
2016-01-07,148.83
2016-01-08,145.56


In [100]:
# Reset index. Then rename columns
aap_closing2016_df.reset_index(inplace=True)
data = aap_closing2016_df.rename(columns = {"index":"date","close_":"close_"})
data.head()

,date,close_
0,2016-01-04,152.24
1,2016-01-05,151.2
2,2016-01-06,147.2
3,2016-01-07,148.83
4,2016-01-08,145.56


In [75]:
# Set a new index to the length of data. Create a new dataframe for date, close_
#aap_closing_data = aal_closing2016_df.rename(columns = {"index":"date","close_":"close_"})
aap_closing_data_df = pd.DataFrame(data)
aap_closing_data_df.head()

,date,close_
0,2016-01-04,152.24
1,2016-01-05,151.2
2,2016-01-06,147.2
3,2016-01-07,148.83
4,2016-01-08,145.56


In [76]:
# Create a dataframe for AAL opening stock price for the year 2016
aal_opening = oneyr16_stocks_df[oneyr16_stocks_df['name_'].isin(['AAL'])]
aal_opening_df = pd.DataFrame(aal_opening)

aal_opening_df.head()


,open_,high,low,close_,volume,name_
2016-01-04,41.29,41.3501,40.3,40.91,12037151,AAL
2016-01-05,41.18,41.49,39.95,40.52,10514164,AAL
2016-01-06,40.05,41.7,40,41.23,12393780,AAL
2016-01-07,40.97,41.52,39.68,40.45,11312876,AAL
2016-01-08,40.94,41.36,40.26,40.37,8853096,AAL


In [77]:
# Create a dataframe for a one yr block (2018)
oneyr18_stocks = stocks_df2.loc['2018-01-01':'2018-12-31']

oneyr18_stocks_df = pd.DataFrame(oneyr18_stocks)
oneyr18_stocks_df.head()

,open_,high,low,close_,volume,name_
2018-01-02,52.33,53.1,51.9,52.99,4084712,AAL
2018-01-03,52.86,52.86,52.055,52.34,5074850,AAL
2018-01-04,52.48,54.4,52.265,52.67,3557059,AAL
2018-01-05,52.78,52.8388,52.43,52.65,2967756,AAL
2018-01-08,52.6,52.64,51.93,52.13,3515785,AAL


In [80]:
# Normalise the data to be processed, In this case the 2016 opening stock price for AAP
from sklearn import preprocessing
data_normaliser = preprocessing.MinMaxScaler()
data_normalised = data_normaliser.fit_transform(aap_closing_data_df.close_.values.reshape(-1, 1))

In [87]:
# Split into features and target arrays. 
y = aap_closing_data_df.close_.values
X = aap_closing_data_df.close_.values.reshape(-1, 1)


# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=80) #, stratify=y)

In [88]:
# Create a StandardScaler instance
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)

# Scale the data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [92]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier


# Train the data
classifier.fit(X_train_scaled, y_train)

# Predict outcomes for test data set
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)



0.0

In [95]:
# Create a random forest classifier.
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")


 Random forest predictive accuracy: 0.063


In [68]:
# Define the model - deep neural network
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  4
hidden_nodes_layer2 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

#  Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#  Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4)                 8         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


In [70]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/1000
189/189 [==============================] - 0s 1ms/sample - loss: -742.6880 - acc: 0.0000e+00
Epoch 2/1000
189/189 [==============================] - 0s 42us/sample - loss: -753.7222 - acc: 0.0000e+00
Epoch 3/1000
189/189 [==============================] - 0s 42us/sample - loss: -764.6180 - acc: 0.0000e+00
Epoch 4/1000
189/189 [==============================] - 0s 58us/sample - loss: -775.7044 - acc: 0.0000e+00
Epoch 5/1000
189/189 [==============================] - 0s 68us/sample - loss: -787.0259 - acc: 0.0000e+00
Epoch 6/1000
189/189 [==============================] - 0s 63us/sample - loss: -798.5101 - acc: 0.0000e+00
Epoch 7/1000
189/189 [==============================] - 0s 37us/sample - loss: -810.1062 - acc: 0.0000e+00
Epoch 8/1000
189/189 [==============================] - 0s 69us/sample - loss: -821.7898 - acc: 0.0000e+00
Epoch 9/1000
189/189 [==============================] - 0s 116us/sample - loss: -833.6862 - acc: 0.0000e+00
Epoch 10/1000
189/189 [==============

Epoch 76/1000
189/189 [==============================] - 0s 74us/sample - loss: -2105.7393 - acc: 0.0000e+00
Epoch 77/1000
189/189 [==============================] - 0s 37us/sample - loss: -2134.2358 - acc: 0.0000e+00
Epoch 78/1000
189/189 [==============================] - 0s 79us/sample - loss: -2162.4650 - acc: 0.0000e+00
Epoch 79/1000
189/189 [==============================] - 0s 48us/sample - loss: -2190.5051 - acc: 0.0000e+00
Epoch 80/1000
189/189 [==============================] - 0s 47us/sample - loss: -2219.8581 - acc: 0.0000e+00
Epoch 81/1000
189/189 [==============================] - 0s 47us/sample - loss: -2249.0277 - acc: 0.0000e+00
Epoch 82/1000
189/189 [==============================] - 0s 53us/sample - loss: -2278.2504 - acc: 0.0000e+00
Epoch 83/1000
189/189 [==============================] - 0s 42us/sample - loss: -2308.3967 - acc: 0.0000e+00
Epoch 84/1000
189/189 [==============================] - 0s 47us/sample - loss: -2338.3931 - acc: 0.0000e+00
Epoch 85/1000
189/1

189/189 [==============================] - 0s 42us/sample - loss: -5197.3877 - acc: 0.0000e+00
Epoch 151/1000
189/189 [==============================] - 0s 42us/sample - loss: -5254.2781 - acc: 0.0000e+00
Epoch 152/1000
189/189 [==============================] - 0s 100us/sample - loss: -5312.6852 - acc: 0.0000e+00
Epoch 153/1000
189/189 [==============================] - 0s 42us/sample - loss: -5372.3233 - acc: 0.0000e+00
Epoch 154/1000
189/189 [==============================] - 0s 47us/sample - loss: -5432.2202 - acc: 0.0000e+00
Epoch 155/1000
189/189 [==============================] - 0s 53us/sample - loss: -5491.7442 - acc: 0.0000e+00
Epoch 156/1000
189/189 [==============================] - 0s 37us/sample - loss: -5553.7207 - acc: 0.0000e+00
Epoch 157/1000
189/189 [==============================] - 0s 32us/sample - loss: -5613.6281 - acc: 0.0000e+00
Epoch 158/1000
189/189 [==============================] - 0s 32us/sample - loss: -5675.9593 - acc: 0.0000e+00
Epoch 159/1000
189/189 [

189/189 [==============================] - 0s 42us/sample - loss: -10939.4725 - acc: 0.0000e+00
Epoch 225/1000
189/189 [==============================] - 0s 95us/sample - loss: -11038.6874 - acc: 0.0000e+00
Epoch 226/1000
189/189 [==============================] - 0s 37us/sample - loss: -11139.2934 - acc: 0.0000e+00
Epoch 227/1000
189/189 [==============================] - 0s 53us/sample - loss: -11239.4250 - acc: 0.0000e+00
Epoch 228/1000
189/189 [==============================] - 0s 47us/sample - loss: -11339.3214 - acc: 0.0000e+00
Epoch 229/1000
189/189 [==============================] - 0s 47us/sample - loss: -11440.8305 - acc: 0.0000e+00
Epoch 230/1000
189/189 [==============================] - 0s 74us/sample - loss: -11542.7197 - acc: 0.0000e+00
Epoch 231/1000
189/189 [==============================] - 0s 42us/sample - loss: -11646.7024 - acc: 0.0000e+00
Epoch 232/1000
189/189 [==============================] - 0s 47us/sample - loss: -11750.2800 - acc: 0.0000e+00
Epoch 233/1000
1

189/189 [==============================] - 0s 42us/sample - loss: -19862.0487 - acc: 0.0000e+00
Epoch 298/1000
189/189 [==============================] - 0s 48us/sample - loss: -20009.6821 - acc: 0.0000e+00
Epoch 299/1000
189/189 [==============================] - 0s 37us/sample - loss: -20157.0298 - acc: 0.0000e+00
Epoch 300/1000
189/189 [==============================] - 0s 47us/sample - loss: -20307.0393 - acc: 0.0000e+00
Epoch 301/1000
189/189 [==============================] - 0s 53us/sample - loss: -20455.4536 - acc: 0.0000e+00
Epoch 302/1000
189/189 [==============================] - 0s 37us/sample - loss: -20603.7974 - acc: 0.0000e+00
Epoch 303/1000
189/189 [==============================] - 0s 42us/sample - loss: -20753.6297 - acc: 0.0000e+00
Epoch 304/1000
189/189 [==============================] - 0s 47us/sample - loss: -20906.1405 - acc: 0.0000e+00
Epoch 305/1000
189/189 [==============================] - 0s 47us/sample - loss: -21056.4607 - acc: 0.0000e+00
Epoch 306/1000
1

...

189/189 [==============================] - 0s 42us/sample - loss: -124464.2173 - acc: 0.0000e+00
Epoch 663/1000
189/189 [==============================] - 0s 58us/sample - loss: -124897.3569 - acc: 0.0000e+00
Epoch 664/1000
189/189 [==============================] - 0s 37us/sample - loss: -125341.9188 - acc: 0.0000e+00
Epoch 665/1000
189/189 [==============================] - 0s 42us/sample - loss: -125777.4206 - acc: 0.0000e+00
Epoch 666/1000
189/189 [==============================] - 0s 48us/sample - loss: -126223.8554 - acc: 0.0000e+00
Epoch 667/1000
189/189 [==============================] - 0s 47us/sample - loss: -126667.4492 - acc: 0.0000e+00
Epoch 668/1000
189/189 [==============================] - 0s 58us/sample - loss: -127099.1095 - acc: 0.0000e+00
Epoch 669/1000
189/189 [==============================] - 0s 48us/sample - loss: -127553.6153 - acc: 0.0000e+00
Epoch 670/1000
189/189 [==============================] - 0s 53us/sample - loss: -127995.3909 - acc: 0.0000e+00
Epoch 6

189/189 [==============================] - 0s 42us/sample - loss: -158863.2507 - acc: 0.0000e+00
Epoch 736/1000
189/189 [==============================] - 0s 53us/sample - loss: -159366.4092 - acc: 0.0000e+00
Epoch 737/1000
189/189 [==============================] - 0s 53us/sample - loss: -159863.5297 - acc: 0.0000e+00
Epoch 738/1000
189/189 [==============================] - 0s 53us/sample - loss: -160377.5257 - acc: 0.0000e+00
Epoch 739/1000
189/189 [==============================] - 0s 47us/sample - loss: -160881.9093 - acc: 0.0000e+00
Epoch 740/1000
189/189 [==============================] - 0s 47us/sample - loss: -161388.5876 - acc: 0.0000e+00
Epoch 741/1000
189/189 [==============================] - 0s 32us/sample - loss: -161901.4630 - acc: 0.0000e+00
Epoch 742/1000
189/189 [==============================] - 0s 42us/sample - loss: -162412.8855 - acc: 0.0000e+00
Epoch 743/1000
189/189 [==============================] - 0s 37us/sample - loss: -162921.6054 - acc: 0.0000e+00
Epoch 7

189/189 [==============================] - 0s 48us/sample - loss: -197617.4702 - acc: 0.0000e+00
Epoch 808/1000
189/189 [==============================] - 0s 47us/sample - loss: -198185.4444 - acc: 0.0000e+00
Epoch 809/1000
189/189 [==============================] - 0s 47us/sample - loss: -198756.6265 - acc: 0.0000e+00
Epoch 810/1000
189/189 [==============================] - 0s 63us/sample - loss: -199352.3574 - acc: 0.0000e+00
Epoch 811/1000
189/189 [==============================] - 0s 84us/sample - loss: -199910.4527 - acc: 0.0000e+00
Epoch 812/1000
189/189 [==============================] - 0s 47us/sample - loss: -200487.5943 - acc: 0.0000e+00
Epoch 813/1000
189/189 [==============================] - 0s 63us/sample - loss: -201071.5637 - acc: 0.0000e+00
Epoch 814/1000
189/189 [==============================] - 0s 42us/sample - loss: -201644.1894 - acc: 0.0000e+00
Epoch 815/1000
189/189 [==============================] - 0s 53us/sample - loss: -202226.5828 - acc: 0.0000e+00
Epoch 8

189/189 [==============================] - 0s 47us/sample - loss: -241356.5605 - acc: 0.0000e+00
Epoch 880/1000
189/189 [==============================] - 0s 53us/sample - loss: -241999.6870 - acc: 0.0000e+00
Epoch 881/1000
189/189 [==============================] - 0s 63us/sample - loss: -242651.2932 - acc: 0.0000e+00
Epoch 882/1000
189/189 [==============================] - 0s 79us/sample - loss: -243280.8018 - acc: 0.0000e+00
Epoch 883/1000
189/189 [==============================] - 0s 64us/sample - loss: -243933.6434 - acc: 0.0000e+00
Epoch 884/1000
189/189 [==============================] - 0s 47us/sample - loss: -244573.2350 - acc: 0.0000e+00
Epoch 885/1000
189/189 [==============================] - 0s 47us/sample - loss: -245230.9688 - acc: 0.0000e+00
Epoch 886/1000
189/189 [==============================] - 0s 47us/sample - loss: -245876.5802 - acc: 0.0000e+00
Epoch 887/1000
189/189 [==============================] - 0s 37us/sample - loss: -246534.3329 - acc: 0.0000e+00
Epoch 8

189/189 [==============================] - 0s 53us/sample - loss: -290264.9696 - acc: 0.0000e+00
Epoch 952/1000
189/189 [==============================] - 0s 53us/sample - loss: -290989.4272 - acc: 0.0000e+00
Epoch 953/1000
189/189 [==============================] - 0s 42us/sample - loss: -291697.7434 - acc: 0.0000e+00
Epoch 954/1000
189/189 [==============================] - 0s 58us/sample - loss: -292417.7583 - acc: 0.0000e+00
Epoch 955/1000
189/189 [==============================] - 0s 53us/sample - loss: -293137.3910 - acc: 0.0000e+00
Epoch 956/1000
189/189 [==============================] - 0s 42us/sample - loss: -293862.5587 - acc: 0.0000e+00
Epoch 957/1000
189/189 [==============================] - 0s 47us/sample - loss: -294582.3403 - acc: 0.0000e+00
Epoch 958/1000
189/189 [==============================] - 0s 47us/sample - loss: -295294.4494 - acc: 0.0000e+00
Epoch 959/1000
189/189 [==============================] - 0s 37us/sample - loss: -296027.6857 - acc: 0.0000e+00
Epoch 9

VISUALIZATION

In [39]:
# Plot closing stock price for AAP(stock name)
#plt.figure(figsize=(12,8))
#plt.plot(aap_opening_df['close_'], label='AAP Close Price history')

In [ ]:
# Create a 3D scatter plot using Plotly Express to plot the clusters using 
# the stocks_df DataFrame.

#fig = px.scatter_3d(
#	  stocks_df,
  x= "open_",
	y="close_",
	z="volume",
  color="date",
	symbol="date",
	width=800,
    hover_name="name_",
    hover_data=["volume"] 
#)
#fig.update_layout(legend=dict(x=0, y=1))
#fig.show()